In [13]:
import pylibxc
import numpy as np
import pandas as pd
from pathlib import Path

cond_to_check = 'lieb_oxford_bd_check_Exc'
xc_or_c = 'xc'
family = 'gga'

file_path = Path(f'{cond_to_check}/{family}_{xc_or_c}.csv')
df = pd.read_csv(file_path, na_values='---')

file = file_path.stem
if xc_or_c == 'xc':
  xc_col = file.replace(f'_{xc_or_c}','').upper() + ' XC'
elif xc_or_c == 'c':
  xc_col = file.replace(f'_{xc_or_c}','').upper() + ' corr.'

df.rename(columns={'xc':xc_col}, inplace=True)


In [14]:
latex_parse = ['@article{', '@article {', '@inproceedings{', '@Inbook{']

bib_file = []
ref_names = []
for xc in df[xc_col]:
  func_c = pylibxc.LibXCFunctional(xc, "polarized")

  ref_name='\cite{'
  for ref in func_c.get_bibtex():
    if ref not in bib_file:
      bib_file.append(ref)
    latex_cite = ref.splitlines()[0]
    for repl in latex_parse:
      latex_cite = latex_cite.replace(repl,'')
    ref_name += latex_cite
    
  ref_name = ref_name[:-1] + '}' 

  ref_names.append(ref_name)

df['Refs.'] = ref_names 

# write out bib file
bib_file = "\n".join(bib_file)
out_file = Path(file_path.parent, file_path.name.replace('.csv','.bib'))
f = open(out_file, "w")
f.write(bib_file)
f.close()

In [15]:
# format functional names
df[xc_col] = df[xc_col].apply(lambda row: row.replace(family+'_c_','').replace('_','\_'))

# format columns
for c in df.columns:
  if 'range' in c:
    df[c] = df[c].apply(lambda row: np.array(row[1:-1].split(', '), dtype=np.float64) if type(row) is str else np.NaN)
    df[c] = df[c].apply(lambda row: np.around(row, decimals=4))

df = df.fillna('---')

In [16]:
with pd.option_context("max_colwidth", 1000):
    latex = df.to_latex(index=False, escape=False, column_format=(len(df.columns)-1)*'|c'+ '|l|')

# horizontal line sep
latex = latex.replace("\\\n", "\\ \hline\n").replace('\\toprule','\\hline')

latex_list = latex.splitlines()
latex=[]
for line in latex_list:
  if '\\toprule' in line or '\\midrule' in line or '\\bottomrule' in line:
    continue
  else:
    latex.append(line)

latex = "\n".join(latex)

out_file = Path(file_path.parent, file_path.name.replace('.csv','.tex'))
f = open(out_file, "w")
f.write(latex)
f.close()

In [17]:
# save updated 
out_file = Path(file_path.parent, file_path.name.replace('.csv','_table.csv'))
df.to_csv(out_file, index=None)